## specifically: stride time (cadence and stride length should be redundant)

In [40]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
#plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

[(0.37254901960784315, 0.48627450980392156, 0.6392156862745098),
 (0.6666666666666666, 0.7647058823529411, 0.8196078431372549),
 (1.0, 0.8235294117647058, 0.44313725490196076),
 (0.37254901960784315, 0.7725490196078432, 0.9254901960784314),
 (0.5372549019607843, 0.803921568627451, 0.4745098039215686),
 (0.8627450980392157, 0.4117647058823529, 0.48627450980392156)]

In [3]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

base_path

PosixPath('/home/rzlin/ys64ofuj/Data/StressGait/Data')

In [4]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=True)

## computation

In [5]:
data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    hs_events = [i for i in hs_events if i > 30]
    stride_times = np.diff(hs_events)
    data.extend([
        (*trial.index.values[0], i, st)
        for i, st in zip(range(len(stride_times)), stride_times)
    ])

## store as multilevel df

In [9]:
stride_df = pd.DataFrame(data, columns=[*dataset.index.columns, "stride_idx", "stride_time"])
stride_df.set_index([*dataset.index.columns, "stride_idx"], inplace=True)
filename = dataset.base_path.joinpath("kinematics/stride_times.csv")
stride_df.to_csv(filename)

In [41]:
stride_df

stride_time
participant condition bout speed stride_idx             
VP_01       omc       1    slow  0                  1.18
                                 1                  1.20
                                 2                  1.24
                                 3                  1.18
                                 4                  1.20
...                                                  ...
VP_47       omc       2    fast  196                1.12
                                 197                1.10
                                 198                1.12
                                 199                1.08
                                 200                1.08

[14942 rows x 1 columns]